In [ ]:
## asynchronously pull the electricity consumption data from 2020-02-02 to 2024-10-31
## Idea is to create a predective model for a day or two from the available dataset
## try to include all the locations and save the data into cloud and model it.

In [3]:
!pip install requests


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
import os
import json
import requests
location_filepath="locations.json"
if not os.path.exists(location_filepath):
    with open(location_filepath,"w",encoding="utf-8") as file:
        response=requests.get("https://helsinki-openapi.nuuka.cloud/api/v1.0/Property/List")
        data=response.text
        json.dump(data,file)

In [18]:
!pip install pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.5 MB 4.9 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/11.5 MB 5.4 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/11.5 MB 2.9 MB/s eta 0:00:04
   ------- -------------------------------- 2.1/11.5 MB 2.4 MB/s eta 0:00:04
   ---------- ----------------------------- 2.9/11.5 MB 2.8 MB/s eta 0:00:04
   ---------- ----------------------------- 3.1/11.5 MB 2.8 MB/s eta 0:00:04
   ----------- ---------------------------- 3.4/11.5 MB 2.6 MB/s eta 0:00:04
   ------------- -------------------------- 3.9/11.5 MB 2.3 MB/s eta 0:00:04
   -------------- ------------------------- 4.2/11.5 MB 2.2 MB/s eta 0:00:04
   ---------------- ----------------------- 4.7/11.5 MB 2.3 MB/s eta 0:00:03
   -------------------- ------------------- 5.8/11.5 MB 2.5 MB/s eta 0:00:03
   --------------------- ------------------ 6.0/11.5 MB 2.5 MB/s eta 0:00:03
   ---


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import pandas as pd

In [23]:
with open(location_filepath, "r", encoding="utf-8") as file:
    # Load the stringified JSON array
    json_content = file.read()
    # Parse the string into a Python object
    data = json.loads(json_content)
    data=json.loads(data)

# Create a pandas DataFrame
df = pd.DataFrame(data)

In [24]:
df.head()

,locationName,propertyName,propertyCode
0,1000 Hakaniemen kauppahalli,1000 Hakaniemen kauppahalli,091-011-9902-0101
1,1001 Hietalahden kauppahalli,1001 Hietalahden kauppahalli,091-004-9902-0008
2,1002 Vanha kauppahalli,1002 Vanha kauppahalli,091-003-9906-0101
3,1037 Vuotalo,1037 Vuotalo,091-054-0179-0003
4,1507 Suutarilan monitoimitalo/ala-aste ja Lpk ...,1507 Suutarilan monitoimitalo/ala-aste ja Lpk ...,091-040-0024-0003


In [25]:
df.shape

(1545, 3)

In [36]:
LocationName=list(df["locationName"][0:4])

In [41]:
dataFileNames = [f"{item.replace(' ', '_')}.json" for item in LocationName]

In [43]:
## locationSearchString=[f"{item.replace(' ', '%20', )}" for item in LocationName]

In [44]:
## locationSearchString

['1000%20Hakaniemen%20kauppahalli',
 '1001%20Hietalahden%20kauppahalli',
 '1002%20Vanha%20kauppahalli',
 '1037%20Vuotalo']

In [42]:
dataFileNames

['1000_Hakaniemen_kauppahalli.json',
 '1001_Hietalahden_kauppahalli.json',
 '1002_Vanha_kauppahalli.json',
 '1037_Vuotalo.json']

In [61]:
reporting_group="Electricity"
start_time="2020-01-01"
end_time="2024-10-31"
version="v1.0"

In [72]:
BASE_URL = "https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Hourly/ListByProperty"
output_dir = "FourlocationData20200101_20241031"
os.makedirs(output_dir, exist_ok=True)


In [53]:
!pip install aiohttp[speedups]



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:
import aiohttp
import asyncio
from urllib.parse import quote

In [63]:
async def fetch_data(session, url, filename):
    async with session.get(url) as response:
        data = await response.json()
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False)
        print(f"Saved data to {filename}")

In [69]:
global count
count=0

In [73]:
async def make_requests():
    async with aiohttp.ClientSession() as session:
        for location in LocationName:
            encoded_search_string = quote(location)
            url = f"{BASE_URL}?Record=LocationName&SearchString={encoded_search_string}&ReportingGroup={reporting_group}&StartTime={start_time}&EndTime={end_time}"
            filename = os.path.join(output_dir,f"{location.replace(' ', '_')}.json")
            await fetch_data(session, url, filename)

await make_requests()

Saved data to FourlocationData20200101_20241031\1000_Hakaniemen_kauppahalli.json
Saved data to FourlocationData20200101_20241031\1001_Hietalahden_kauppahalli.json
Saved data to FourlocationData20200101_20241031\1002_Vanha_kauppahalli.json
Saved data to FourlocationData20200101_20241031\1037_Vuotalo.json
